In [9]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.path.abspath('')).parent))
print(str(Path(os.path.abspath('')).parent))
import math
import torch
from torch import Tensor
from delaunay_rasterization.internal.alphablend_tiled_slang import render_alpha_blend_tiles_slang_raw
from scipy.spatial import Voronoi, Delaunay
from torch import nn
import matplotlib.pyplot as plt
import mediapy
from icecream import ic
from data import loader
import random
import time
import tinycudann as tcnn
from utils.contraction import contract_mean_std
from utils import topo_utils
from tqdm import tqdm
import numpy as np
# from dtet import DelaunayTriangulation
from dtet.build.dtet import DelaunayTriangulation

K = 20

def fov2focal(fov, pixels):
    return pixels / (2 * math.tan(fov / 2))

def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def get_slang_projection_matrix(znear, zfar, fy, fx, height, width, device):
    tanHalfFovX = width/(2*fx)
    tanHalfFovY = height/(2*fy)

    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    z_sign = 1.0

    P = torch.tensor([
       [2.0 * znear / (right - left),     0.0,                          (right + left) / (right - left), 0.0 ],
       [0.0,                              2.0 * znear / (top - bottom), (top + bottom) / (top - bottom), 0.0 ],
       [0.0,                              0.0,                          z_sign * zfar / (zfar - znear),  -(zfar * znear) / (zfar - znear) ],
       [0.0,                              0.0,                          z_sign,                          0.0 ]
    ], device=device)

    return P

def common_camera_properties_from_gsplat(viewmats, Ks, height, width):
  """ Fetches all the Camera properties from the inria defined object"""
  zfar = 100.0
  znear = 0.01
  
  world_view_transform = viewmats
  fx = Ks[0,0]
  fy = Ks[1,1]
  projection_matrix = get_slang_projection_matrix(znear, zfar, fy, fx, height, width, Ks.device)
  fovx = focal2fov(fx, width)
  fovy = focal2fov(fy, height)

  cam_pos = viewmats.inverse()[:, 3]

  return world_view_transform, projection_matrix, cam_pos, fovy, fovx

/home/amai/delaunay_rasterization


In [10]:
train_cameras, test_cameras, scene_info = loader.load_dataset("/data/nerf_datasets/360/bicycle", "images_8", data_device="cuda", eval=True)

Reading camera 194/194
Loaded Train Cameras: 169
Loaded Test Cameras: 25


In [11]:

torch.manual_seed(2)
N = scene_info.point_cloud.points.shape[0]
vertices = torch.as_tensor(scene_info.point_cloud.points)[:N]
minv = vertices.min(dim=0, keepdim=True).values
maxv = vertices.max(dim=0, keepdim=True).values
repeats = 10
vertices = vertices.reshape(-1, 1, 3).expand(-1, repeats, 3)
vertices = vertices + torch.randn(*vertices.shape) * 1e-1
N = 50000
vertices = torch.cat([
  vertices.reshape(-1, 3),
  torch.rand((N, 3)) * (maxv - minv) + minv
], dim=0)
vertices = nn.Parameter(vertices.cuda())

device = torch.device('cuda')
encoding = tcnn.Encoding(3, dict(
    otype="HashGrid",
    n_levels=16,
    n_features_per_level=2,
    log2_hashmap_size=14,
    base_resolution=1,
    per_level_scale=1.5
))
network = tcnn.Network(encoding.n_output_dims, 4, dict(
    # otype="CutlassMLP",
    otype="FullyFusedMLP",
    activation="ReLU",
    output_activation="None",
    n_neurons=64,
    n_hidden_layers=2,
))
net = torch.nn.Sequential(
    encoding, network
).to(device)


def safe_exp(x):
  return x.clip(max=5).exp()

def safe_trig_helper(x, fn, t=100 * torch.pi):
  """Helper function used by safe_cos/safe_sin: mods x before sin()/cos()."""
  return fn(torch.nan_to_num(torch.where(torch.abs(x) < t, x, x % t)))


def safe_cos(x):
  """jnp.cos() on a TPU may NaN out for large values."""
  return safe_trig_helper(x, torch.cos)


def safe_sin(x):
  """jnp.sin() on a TPU may NaN out for large values."""
  return safe_trig_helper(x, torch.sin)

def rgbs_fn(xyz):
  cxyz, _ = contract_mean_std(xyz, torch.ones_like(xyz[..., 0]))
  rgbs_raw = net((cxyz/2 + 1)/2).float()
  rgbs = torch.cat([torch.sigmoid(rgbs_raw[:, :3]), safe_exp(rgbs_raw[:, 3:]-3)], dim=1)
  return rgbs


In [12]:
camera = train_cameras[0]
print(camera.projection_matrix)

tensor([[ 1.8801,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  2.8164,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0001,  1.0000],
        [ 0.0000,  0.0000, -0.0100,  0.0000]], device='cuda:0')


In [13]:
def render(camera, indices, vertices, cell_values=None):
    fy = fov2focal(camera.fovy, camera.image_height)
    fx = fov2focal(camera.fovx, camera.image_width)
    K = torch.tensor([
    [fx, 0, camera.image_width/2],
    [0, fy, camera.image_height/2],
    [0, 0, 1],
    ]).cuda()

    # world_view_transform, projection_matrix, cam_pos, fovy, fovx = common_camera_properties_from_gsplat(
    #     camera.world_view_transform.T, K, camera.image_height, camera.image_width)
    cam_pos = camera.world_view_transform.T.inverse()[:, 3]

    render_pkg = render_alpha_blend_tiles_slang_raw(indices, vertices, rgbs_fn,
                                                    camera.world_view_transform.T, K, cam_pos,
                                                    camera.fovy, camera.fovx, camera.image_height, camera.image_width,
                                                    cell_values=cell_values, tile_size=tile_size)
    return render_pkg

optim = torch.optim.Adam([
    {"params": net.parameters(), "lr": 1e-2},
    {"params": [vertices], "lr": 2.5e-4},
])
tile_size = 8
images = []
for i in tqdm(range(5001)):


    ind = random.randint(0, len(train_cameras)-1)
    camera = train_cameras[ind]

    # st = time.time()
    if i % 20 == 0:
        if (i > 0):
            loss.backward()
            optim.step()
            optim.zero_grad()
        loss = 0
        render_pkg = render(camera, indices, vertices)
        circumcenter = render_pkg['circumcenters']
        rgbs = torch.zeros((circumcenter.shape[0], 4), device=circumcenter.device)
        rgbs[:indices.shape[0]] = rgbs_fn(circumcenter[:indices.shape[0]])

        ind = 0
        camera = train_cameras[ind]
        render_pkg = render(camera, indices, vertices, cell_values=rgbs)
        image = render_pkg['render']
        image = image.permute(1, 2, 0)
        image = image.detach().cpu().numpy()
        images.append(image)

    if i % 20 == 0:
        # v = Delaunay(vertices.detach().cpu().numpy())
        # indices_np = v.simplices
        # indices_np = indices_np[np.lexsort(indices_np.T)]

        v = DelaunayTriangulation()
        v.init_from_points(vertices.detach().cpu().numpy())
        indices_np = v.get_cells().astype(np.int32)

        indices = torch.as_tensor(indices_np).cuda()
        old_vertices = vertices.detach().cpu().numpy()

    target = camera.original_image.cuda()

    st = time.time()
    render_pkg = render(camera, indices, vertices, cell_values=rgbs)
    # print(1/(time.time()-st))
    image = render_pkg['render']
    loss += ((target - image)**2).mean()

    # new_vertices = vertices.detach().cpu().numpy()
    # indices_np, _ = topo_utils.update_tetrahedralization(old_vertices, new_vertices, indices_np)
    # indices = torch.as_tensor(indices_np).cuda()

    if i % 200 == 0:
        # plt.imshow(image)
        # plt.show()
        mediapy.show_video(images)
mediapy.show_video(images)

  0%|          | 0/5001 [00:00<?, ?it/s]

"<source src=""data:video/mp4;base64,AAAAIGZ0eXBpc29tAAACAGlzb21pc28yYXZjMW1wNDEAAAAIZnJlZQAABQZtZGF0AAACfwYF//973EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2NCByMzA5NSBiYWVlNDAwIC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAyMiAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD00IHRocmVhZHM9MTMgbG9va2FoZWFkX3RocmVhZHM9MiBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxhY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHlyYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3ZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTI1IHNjZW5lY3V0PTQwIGludHJhX3JlZnJlc2g9MCByYz1jcXAgbWJ0cmVlPTAgcXA9MjAgaXBfcmF0aW89MS40MCBwYl9yYXRpbz0xLjMwIGFxPTAAgAAAAndliIQA//7zm+BRLQ/ypT+xVnX/k1Np5LzAAAYhyLBi8v87w65b2RhiWZ+8CPLlfmY+Mk3paAqDWBA/wgigqSUsCFAkfNi1lFG8+371VEhDcnGfcia0z4mnioMNbvYXOcNKbpfDST5gm6nyuBtFeajWbzEfGKRI5j8SfH8M1p0yrRdb6TuPF6/N2rYNX5joHBEVpUtECKDq2pdLm+ouAk4o2o5Pqdzg4KN7j9mi0BiMDqvsDoYyuBfEoPTPFQaV09AcVYF3uch4fQtCBdMLh40lFIJkqNOHYsZvLMfmJAN8vSbwdFKQr6drTS//Nrg7+SRTqQ2zMgJwQzhblVLmGCe0GvGUcuL/k46rGpj8uXErbpYmK6hDBX41qbq6mVyJ695pTB5sklcTETi6uG6MA3bZ7tJPHNJLcJ9rPwJi6aUI6Dry4ZVvlOY1SLoRrGGUMKqvz44EBqZ6iIndSUHySqC7UBIRyGgIk0ipOkfrFjsiOKZ8RDoY/42bvaIJyZsZMft04QGG3XVwyby01b0WcU6/N07JtwFzQAAAAwIj98TN2qDAEhT6GREJ8hX/uy4x6vCY7t5WLVpAaWUVZS6hoNI+W2/DhU4nPhW9XPJRPWW+5fxN5CWZLujbt0DwVNqzeqnNjUw299gKC0FEcwGHavOtPrguiIRTq+Q0p5PydRPcJ9k6oXYsr+VLQnFAMQnBhwAMQ4o2B4LdR/11Ck+hSuvm8dwwPNZabaoVocbAHLjY3RvcEMHWMJMmIiMonXozTxJstZE/tn/277oCb2/sFuQD86aponIhoxNLWJtCjRuIgM+t1Brgn9nYwRteAC0n97TRF8r49RaJAAADB21vb3YAAABsbXZoZAAAAAAAAAAAAAAAAAAAA+gAAAARAAEAAAEAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAIydHJhawAAAFx0a2hkAAAAAwAAAAAAAAAAAAAAAQAAAAAAAAARAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAQAAAAAJqAAABmwAAAAAAJGVkdHMAAAAcZWxzdAAAAAAAAAABAAAAEQAAAAAAAQAAAAABqm1kaWEAAAAgbWRoZAAAAAAAAAAAAAAAAAAAPAAAAAEAVcQAAAAAAC1oZGxyAAAAAAAAAAB2aWRlAAAAAAAAAAAAAAAAVmlkZW9IYW5kbGVyAAAAAVVtaW5mAAAAFHZtaGQAAAABAAAAAAAAAAAAAAAkZGluZgAAABxkcmVmAAAAAAAAAAEAAAAMdXJsIAAAAAEAAAEVc3RibAAAALFzdHNkAAAAAAAAAAEAAAChYXZjMQAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAJqAZsASAAAAEgAAAAAAAAAARVMYXZjNjEuMTkuMTAwIGxpYngyNjQAAAAAAAAAAAAAABj//wAAADdhdmNDAfQAH//hABpn9AAfkZsoE4a880IAAAMAAgAAAwDwHjBjLAEABmjr4bESEf/4+AAAAAAUYnRydAAAAAAACVxAAAlcQAAAABhzdHRzAAAAAAAAAAEAAAABAAABAAAAABxzdHNjAAAAAAAAAAEAAAABAAAAAQAAAAEAAAAUc3RzegAAAAAAAAT+AAAAAQAAABRzdGNvAAAAAAAAAAEAAAAwAAAAYXVkdGEAAABZbWV0YQAAAAAAAAAhaGRscgAAAAAAAAAAbWRpcmFwcGwAAAAAAAAAAAAAAAAsaWxzdAAAACSpdG9vAAAAHGRhdGEAAAABAAAAAExhdmY2MS43LjEwMA=="" type=""video/mp4""/> This browser does not support the video tag."


  4%|▍         | 188/5001 [00:41<19:20,  4.15it/s]

  8%|▊         | 400/5001 [01:25<12:12,  6.28it/s]

This browser does not support the video tag.


...

This browser does not support the video tag.


 68%|██████▊   | 3397/5001 [12:22<04:29,  5.95it/s]

This browser does not support the video tag.


 70%|██████▉   | 3480/5001 [12:42<05:33,  4.56it/s]


KeyboardInterrupt: 